In [ ]:
!pip install google-oauth2-tool
!pip install google-api-python-client

Import 必須的套件

In [ ]:
import requests
import json
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import os

這邊要GCP service account 的credential

# Locate the json key file generated in the previous step
KEY_FILE_LOCATION = 'yout_credential_path'

In [ ]:
def connect(KEY_FILE_LOCATION):
    """Create a connection to the Google Search Console API and return service object.

    Args:
        key (string): Google Search Console JSON client secrets path.

    Returns:
        service (object): Google Search Console service object.
    """

    scope = ['https://www.googleapis.com/auth/webmasters.readonly']
    credentials = service_account.Credentials.from_service_account_file(KEY_FILE_LOCATION, scopes=scope)
    service = build('searchconsole', 'v1',
        credentials=credentials
    )

    return service


def query(service, site_url, payload):
    """Run a query on the Google Search Console API and return a dataframe of results.

    Args:
        service (object): Service object from connect()
        site_url (string): URL of Google Search Console property
        payload (dict): API query payload dictionary

    Return:
        df (dataframe): Pandas dataframe containing requested data.

    """

    response = service.searchanalytics().query(siteUrl=site_url, body=payload).execute()

    results = []

    for row in response['rows']:
        data = {}

        for i in range(len(payload['dimensions'])):
            data[payload['dimensions'][i]] = row['keys'][i]

        data['clicks'] = row['clicks']
        data['impressions'] = row['impressions']
        data['ctr'] = round(row['ctr'] * 100, 2)
        data['position'] = round(row['position'], 2)
        results.append(data)

    return pd.DataFrame.from_dict(results)

In [ ]:
service = connect(KEY_FILE_LOCATION)

payload = {
    'startDate': "2023-07-01",
    'endDate': today,
    'dimensions': ["page","device","query"],
    'rowLimit': 25000,
    'startRow': 0
}

site_url = "https://www.tcloud.gov.tw/"

df = query(service, site_url, payload)
df.head(10)

匯出csv

In [ ]:
# 假設您的DataFrame名稱為df，檔案名稱為output.csv
df.to_csv('searchconsole_july.csv', index=False)